In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
BATCH_SIZE = 1
MAX_LENGTH = 256
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
forget_texts = ["Biologie et génétique des cellules humaines."]
retain_texts = ["La révolution française commence en 1789."]
print(":rocket: Chargement du modèle Mixtral (frozen + updated)...")
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"
frozen_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
updated_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"

def forward_with_cache(model, inputs, module):
    cache = []
    def hook(module, input, output):
        cache.append(output[0] if isinstance(output, tuple) else output)
    handle = module.register_forward_hook(hook)
    with torch.no_grad():
        _ = model(**inputs)
    handle.remove()
    return cache[0]
forget_inputs = tokenizer(forget_texts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LENGTH)
forget_inputs = {k: v.to(DEVICE) for k, v in forget_inputs.items()}
retain_inputs = tokenizer(retain_texts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LENGTH)
retain_inputs = {k: v.to(DEVICE) for k, v in retain_inputs.items()}
layer_id = 7
updated_module = updated_model.model.layers[layer_id]
frozen_module = frozen_model.model.layers[layer_id]
print(":arrows_counterclockwise: Forward forget")
forget_acts = forward_with_cache(updated_model, forget_inputs, updated_module)
print(f":white_check_mark: forget shape: {forget_acts.shape} | dtype: {forget_acts.dtype}")
print(":arrows_counterclockwise: Forward retain")
retain_acts = forward_with_cache(frozen_model, retain_inputs, frozen_module)
print(f":white_check_mark: retain shape: {retain_acts.shape} | dtype: {retain_acts.dtype}")
print(f":brain: VRAM utilisée : {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f":brain: VRAM max : {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

KeyboardInterrupt: 

In [1]:
# best
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5"

!python3 -m rmu.unlearn_WMDP \
  --model_name mistralai/Mixtral-8x7B-Instruct-v0.1  \
  --batch_size 1 \
  --max_num_batches 20 \
  --param_ids 7 \
  --retain_corpora wikitext,wikitext \
  --forget_corpora bio-forget-corpus,cyber-forget-corpus \
  --steering_coeffs 300,300 \
  --alpha 1600,1600 \
  --min_len 200 \
  --lr 5e-5 \
  --seed 42 \
  --output_dir models/mixtral_rmu \
  --verbose



Loading checkpoint shards:  21%|███▌             | 4/19 [00:38<02:28,  9.90s/it]

In [2]:
"""Other pipelines we tested"""
""" !python3 -m rmu.unlearn \
  --model_name mistralai/Mixtral-8x7B-Instruct-v0.1 \
  --batch_size 1 \
  --param_ids 7 \
  --max_num_batches 200 \
  --retain_corpora retain_qa,retain_qa_icl \
  --forget_corpora forget_qa,forget_qa_icl \
  --steering_coeffs 300 \
  --alpha 1600 \
  --min_len 200 \
  --lr 5e-5 \
  --seed 42 \
  --output_dir models/mixtral_muse_rmu \
  --verbose """
""" !python -m rmu.unlearn \
  --model_name_or_path mistralai/Mixtral-8x7B-Instruct-v0.1 \
  --batch_size 1 \
  --param_ids 7 \
  --max_num_batches 200 \
  --retain_corpora retain_qa,retain_qa_icl \
  --forget_corpora forget_qa,forget_qa_icl \
  --steering_coeffs 300,300 \
  --alpha 1600,1600 \
  --min_len 200 \
  --lr 5e-5 \
  --seed 42 \
  --output_dir models/mixtral_muse_rmu \
  --verbose """
""" !python -m rmu.unlearn \
  --model_name_or_path "microsoft/DialoGPT-medium" \
  --forget_corpora "forget_qa" \
  --retain_corpora "retain_qa" \
  --alpha "30" \
  --steering_coeffs "10" \
  --lr 5e-5 \
  --max_num_batches 200 \
  --layer_id 12 \
  --verbose """

' !python -m rmu.unlearn   --model_name_or_path "microsoft/DialoGPT-medium"   --forget_corpora "forget_qa"   --retain_corpora "retain_qa"   --alpha "30"   --steering_coeffs "10"   --lr 5e-5   --max_num_batches 200   --layer_id 12   --verbose '

In [3]:
""" !lm-eval --model hf \
    --model_args pretrained=models/mixtral_rmu,parallelize=True \
    --tasks wmdp,mmlu \
    --batch_size=32 """
"""!lm-eval \
  --model hf \
  --model_args pretrained=mistralai/Mixtral-8x7B-Instruct-v0.1,use_auth_token=True \
  --tasks mmlu,wmdp \
  --output_path results/mixtral_results.json"""
"""!lm-eval \
  --model hf \
  --model_args pretrained=EleutherAI/gpt-neo-1.3B,dtype=float16,device=cuda \
  --tasks mmlu,wmdp \
  --output_path results/gptneo_results.json"""
"""!lm-eval \
  --model hf \
  --model_args pretrained=tiiuae/falcon-7b-instruct,dtype=float16,device=cuda \
  --tasks mmlu,wmdp \
  --output_path results/falcon_results.json"""
"""!lm-eval \
  --model hf \
  --model_args pretrained=mistralai/Mixtral-8x7B-Instruct-v0.1,use_auth_token=True,dtype=float16 \
  --tasks mmlu,wmdp \
  --output_path results/mixtral_results.json"""
""" !lm-eval \
  --model hf \
  --model_args pretrained=mistralai/Mixtral-8x7B-Instruct-v0.1,use_auth_token=True,device=cpu \
  --tasks mmlu,wmdp \
  --output_path results/mixtral_results.json """
"""!lm-eval \
  --model hf \
  --model_args pretrained=mistralai/Mixtral-8x7B-Instruct-v0.1,use_auth_token=True,device=cpu \
  --tasks mmlu,wmdp \
  --output_path results/mixtral_results.json \
  --trust_remote_code"""
"""!lm-eval \
  --model hf \
  --model_args pretrained=tiiuae/falcon-7b-instruct,dtype=float16,device=cuda \
  --tasks mmlu,wmdp \
  --output_path results/falcon7b_results.json \
  --trust_remote_code"""
""" !export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True && \
lm-eval \
  --model hf \
  --model_args pretrained=tiiuae/falcon-7b-instruct,token=True,dtype=float16,device=cuda \
  --tasks mmlu,wmdp \
  --output_path results/falcon7b_results.json \
  --trust_remote_code   """
"""!lm-eval \
  --model hf \
  --model_args pretrained=tiiuae/falcon-7b-instruct,token=True,dtype=float16,device=cuda \
  --tasks mmlu,wmdp \
  --output_path results/falcon7b_results.json \
  --trust_remote_code \
  --batch_size 1"""
# Working for WMDP
!lm-eval \
  --model hf \
  --model_args pretrained=tiiuae/falcon-7b-instruct,load_in_8bit=True,token=True,device=cuda \
  --tasks mmlu,wmdp \
  --output_path results/eval_results.json \
  --trust_remote_code \
  --batch_size 1 \
  --limit 1000

""" !export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True && \
lm-eval \
  --model hf-causal-experimental \
  --model_args pretrained=./models/mixtral_muse_rmu,trust_remote_code=True \
  --tasks muse_knowmem,muse_privleak \
  --output_path results/mixtral_muse_eval.json \
  --trust_remote_code \
  --batch_size 1 \
  --limit 1000 """
#!lm-eval --tasks list | grep muse
""" !lm-eval \
  --model hf \
  --model_args pretrained=models/mixtral_muse_rmu \
  --tasks muse_knowmem,muse_privleak \
  --num_fewshot 0 \
  --batch_size 1 \
  --output_path results/mixtral_muse_rmu_eval.json \
  --limit 1000 """
""" !lm-eval --model hf \
  --model_args pretrained=models/mixtral_muse_rmu \
  --tasks muse_knowmem \
  --device cuda \
  --batch_size 1 """
""" !export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True && \
lm-eval \
  --model hf \
  --model_args "pretrained=models/mixtral_muse_rmu,load_in_4bit=true,bnb_4bit_compute_dtype=float16,device_map=auto,trust_remote_code=true" \
  --batch_size 1 \
  --tasks muse_knowmem \
  --limit 1000 """
""" !lm-eval \
  --model hf \
  --model_args "pretrained=models/mixtral_muse_rmu,load_in_4bit=true,bnb_4bit_compute_dtype=float16,bnb_4bit_use_double_quant=true,device_map=auto,trust_remote_code=true" \
  --batch_size 1 \
  --tasks muse_knowmem \
  --limit 1000 """
""" !lm-eval \
  --model hf \
  --model_args "pretrained=models/mixtral_muse_rmu,load_in_4bit=true,bnb_4bit_compute_dtype=float16,bnb_4bit_use_double_quant=true,bnb_4bit_quant_type=nf4,low_cpu_mem_usage=true,device_map=auto,trust_remote_code=true" \
  --batch_size 1 \
  --tasks muse_knowmem \
  --limit 1000 """
""" !lm-eval \
  --model hf \
  --model_args "pretrained=models/mixtral_muse_rmu,load_in_4bit=true,bnb_4bit_compute_dtype=float16,bnb_4bit_use_double_quant=true,bnb_4bit_quant_type=nf4,low_cpu_mem_usage=true,device_map=auto,trust_remote_code=true" \
  --batch_size 1 \
  --tasks muse_knowmem \
  --limit 1000 """

2025-06-30:23:41:54 WARNING  [__main__:368]  --limit SHOULD ONLY BE USED FOR TESTING.REAL METRICS SHOULD NOT BE COMPUTED USING LIMIT.
2025-06-30:23:41:54 INFO     [__main__:428] Passed `--trust_remote_code`, setting environment variable `HF_DATASETS_TRUST_REMOTE_CODE=true`
2025-06-30:23:41:54 INFO     [__main__:440] Selected Tasks: ['mmlu', 'wmdp']
2025-06-30:23:41:54 WARNING  [evaluator:163] Model appears to be an instruct variant but chat template is not applied. Recommend setting `apply_chat_template` (optionally `fewshot_as_multiturn`).
2025-06-30:23:41:54 INFO     [evaluator:189] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-06-30:23:41:54 INFO     [evaluator:227] Initializing hf model, with arguments: {'pretrained': 'tiiuae/falcon-7b-instruct', 'load_in_8bit': True, 'token': True, 'device': 'cuda', 'trust_remote_code': True}
2025-06-30:23:41:54 INFO     [models.huggingface:137] Using device 'cu

' !lm-eval   --model hf   --model_args "pretrained=models/mixtral_muse_rmu,load_in_4bit=true,bnb_4bit_compute_dtype=float16,bnb_4bit_use_double_quant=true,bnb_4bit_quant_type=nf4,low_cpu_mem_usage=true,device_map=auto,trust_remote_code=true"   --batch_size 1   --tasks muse_knowmem   --limit 1000 '